In [3]:
import datajoint as dj

print("neuron tab labeler started new")
#setting the address and the username
print("about to connect to database")
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
#will state whether words are shown or not
dj.config['safemode']=True
print(dj.conn(reset=True))


AttributeError: type object 'numpy.ndarray' has no attribute '__array_function__'

In [ ]:
#connect_to_Databases()
#create the database inside the server
schema = dj.schema('microns_ta3',create_tables=False)
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
nda = dj.create_virtual_module('nda2', 'microns_nda2')
import numpy as np
#reset_Scene_Variables()

In [ ]:
#gets all of the axon segment ids         
axon_segments = (ta3p100.Neurite() & 'neurite_type="axon"' & ta3p100.CurrentSegmentation).proj(presyn='segment_id')
axon_segments

In [ ]:
#get all of the post-synaptic segments that are significantly tuned

########*******DON'T NEED TO RESTRICT BY TRACE ANYMORE ********#####  so just getting the ones with somas
#####Needs to be the same ones we have the annotations for
#post_trace_segments = (ta3p100.Segment & ta3p100.AllenSoma).proj(postsyn = 'segment_id')
post_trace_segments = (ta3p100.Segment & ta3p100.ComponentLabel).proj(postsyn = 'segment_id')
post_trace_segments

In [ ]:
synapse_ids_error = (ta3p100.SynapseCompartmentLabel() & "segmentation=2" & "postsynaptic_label=10")

In [ ]:
#need to filter away all of the errors and no color
final_synapses_with_labels = ((ta3p100.Synapse() & post_trace_segments & axon_segments)) * ta3p100.SynapseCompartmentLabel
final_synapses_with_labels

In [ ]:
#need to filter away all of the errors and no color
final_synapses_with_labels = ((ta3p100.Synapse() & post_trace_segments & axon_segments) - ta3p100.SynapseExclude.proj() - synapse_ids_error.proj()) * ta3p100.SynapseCompartmentLabel
final_synapses_with_labels


In [ ]:
#calculate the distance based on the table
synapses_only_distance = (final_synapses_with_labels.proj("presyn","postsyn","synapse_x","synapse_y","synapse_z","postsynaptic_label"
     ,synaptic_size="(syn_bbox_x2 - syn_bbox_x1)*(syn_bbox_y2- syn_bbox_y1)*(syn_bbox_z2-syn_bbox_z1)"))

synapses_only_distance





In [ ]:
#ta3.Soma()
soma_info = ta3p100.AllenSoma().proj("soma_x","soma_y","soma_z",postsyn = "segment_id")

In [ ]:
#shorten the table by computing the distances and the norm
complete_synapse_table = ((synapses_only_distance * soma_info * ta3.LabelKey().proj('description', postsynaptic_label='numeric')).proj("presyn","synaptic_size","postsynaptic_label","description"
            ,x_dist='synapse_x-soma_x',y_dist='synapse_y-soma_y',
            norm='sqrt(POWER((synapse_x-soma_x),2) + POWER((synapse_y-soma_y),2) + POWER((synapse_z-soma_z),2)  )'))


complete_synapse_table #& 'description_spine="Spine"'





In [ ]:
complete_synapse_table_fixed = ((dj.U("segmentation","synapse_id","presyn","postsyn","postsynaptic_label") & complete_synapse_table) 
            *  complete_synapse_table.proj("synaptic_size","description","x_dist","y_dist","norm"))
                                
complete_synapse_table_fixed       



In [ ]:
"""   HOW THE SharedInput Table is generated
#how they generate the SharedInput
soma = ta3.Segment & nda.Trace
axons = ta3.Neurite & 'neurite_type="axon"'
synapse = ta3.Synapse & axons.proj(presyn='segment_id') & complete_synapse_table_fixed
info = (soma * soma.proj(segment_b='segment_id') & 'segment_id > segment_b' 
                & complete_synapse_table_fixed.proj(segment_b="postsyn") 
                & complete_synapse_table_fixed.proj(segment_id="postsyn"))
A = (synapse & soma.proj(postsyn='segment_id')).proj(
    'presyn', syn1='synapse_id', segment_id='postsyn')
B = (synapse * dj.U('presyn') & soma.proj(postsyn='segment_id')).proj(
    'presyn', syn2='synapse_id', segment_b='postsyn')
shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id > segment_b',
                                              n_seg_shared='count(DISTINCT presyn)')



shared
other_half_table = #info - shared """

In [ ]:
#pull down the data
synapse_id,postsyn,postsynaptic_label,description,presyn = complete_synapse_table.fetch("synapse_id",
                                                        "postsyn","postsynaptic_label","description","presyn")



In [ ]:
#add on the spine labels
synapse_id_spine, spine_label = ta3p100.SynapseSpineLabel().fetch("synapse_id","spine_label")



In [ ]:
label_key = ta3.LabelKey().fetch("description")
label_key[0]

In [ ]:
from tqdm import tqdm
#create lookup index
spine_lookup=dict()
for syn_id, syn_label in tqdm(zip(synapse_id_spine,spine_label)):
    spine_lookup[syn_id] = syn_label
    
#find out who is not in the list
#print(spine_lookup)

In [ ]:

#create the extra spine list
postsynaptic_label_spine = list()
spine_description = list()
spine_label_lookup = dict()
synapse_id_spine = set(synapse_id_spine)
for i,syn in tqdm(enumerate(synapse_id)):
    if syn in synapse_id_spine:
        #print("inside labeler")
        label = spine_lookup[syn]
    else:
        label = 0
        
    spine_label_lookup[syn] = label
    postsynaptic_label_spine.append(label)
    spine_description.append(label_key[label]) 

        
        

In [ ]:
#create pandas table from all of the data
import pandas as pd
postsynaptic_label_spine

synapse_id,postsyn,postsynaptic_label,description,presyn
d = dict(synapse_id=synapse_id,presyn=presyn,postsyn=postsyn,postsynaptic_label=postsynaptic_label,
         spine_numeric=postsynaptic_label_spine,description=description,spine_description=spine_description
          )

Label_Table = pd.DataFrame(d)
Label_Table



In [ ]:
#query the datatable for list
possible_spines = ["Spine","Spine Neck","Spine Head"]
#possible_spines = ["not_labeled"]
Label_Table.loc[Label_Table["spine_description"].isin(possible_spines)]




In [ ]:
#find the synapses without the Spines
no_spine_table = Label_Table.loc[Label_Table["spine_description"].isin(["not_labeled","Spine Neck","Spine Head"])]
no_spine_table

In [ ]:
from collections import Counter
#pull out of the dataframe, compute the statistics and then put back in the table
#make sure these are numpy arrays or else the where function won't work
synapse_ids = no_spine_table['synapse_id']
presyns = Label_Table['presyn']
postsyns = Label_Table['postsyn']
synapse_labels = Label_Table['postsynaptic_label']
synapse_labels_spine = no_spine_table['spine_numeric']
description = Label_Table['description']
description_spine = Label_Table['spine_description']



''' Way Christos aggregates:
1) Creates counter for pre and post
2) creates dictionary that maps the postsynaptic target to the synapse
3) creates dictionary for synap --> label
    create reg label, joint label, and spine label
4) maps axons --> synapse
'''


"""
#aggregate the data
postsyns_to_synapses = dict()
for postsyn in tqdm(postsyns_counter):
    postsyns_to_synapses[postsyn] = synapse_ids[np.where(postsyns==postsyn)[0]]
    
#creates dictionary for synap --> label
synapse_label_dict = dict()
for synapse_id, label in tqdm(zip(*synapse_labels)):
    synapse_label_dict[synapse_id] = label

#maps axons --> synapse
axon_to_synapse = dict()
for synapse, presyn in tqdm(zip(synapse_ids, presyns)):
    if presyn in axon_to_synapse:
        axon_to_synapse[presyn].append(synapse)
    else:
        axon_to_synapse[presyn] = list()
        axon_to_synapse[presyn].append(synapse)
        
"""

In [ ]:
compartment_labels = Counter(description).keys()
compartment_labels

In [ ]:
#gets the counter for presynapses and postysnaptic
presyns_counter, postsyns_counter = Counter(presyns), Counter(postsyns)
#print(presyns_counter)

#creates dictionary that maps the postsynaptic target to the synapse

postsyns_to_synapses = dict()
for postsyn in tqdm(postsyns_counter.keys()):
    postsyns_to_synapses[postsyn] = synapse_ids[np.where(postsyns==postsyn)[0]]
    
#creates dictionary for synap --> label
synapse_label_dict = dict()
for synapse_id, label in tqdm(zip(synapse_ids,synapse_labels)):
    synapse_label_dict[synapse_id] = label

#creates dictionary for synap --> spine label
synapse_label_dict_spine = dict()
for synapse_id, label in tqdm(zip(synapse_ids,synapse_labels_spine)):
    synapse_label_dict_spine[synapse_id] = label
    
#DON'T THINK NEED JOINT LABELS!!
#maps axons --> synapse
axon_to_synapse = dict()
for synapse, presyn in tqdm(zip(synapse_ids, presyns)):
    if presyn in axon_to_synapse:
        axon_to_synapse[presyn].append(synapse)
    else:
        axon_to_synapse[presyn] = list()
        axon_to_synapse[presyn].append(synapse)



In [ ]:
#The destination vectors that will be fed into the clustering machine

#WHERE COMPARTMENT LABELS AND SPINES NOT SEPERATE

axon_to_vector = dict()
threshold_axon_to_vector = dict()
axon_to_vector_spine = dict()
threshold_axon_to_vector_spine = dict()

axon_to_vector_spine_joint = dict()
threshold_axon_to_vector_spine_joint = dict()



#aggregating the data
for presyn in tqdm(axon_to_synapse):
    label_list = list()
    label_list_spine = list()
    #list to hold both compartments
    joint_list = list()
    for synapse in axon_to_synapse[presyn]:
        label_list.append(synapse_label_dict[synapse])
        label_list_spine.append(synapse_label_dict_spine[synapse])
        joint_list.append(synapse_label_dict[synapse]*100 + synapse_label_dict_spine[synapse])
    label_counter = Counter(label_list)
   
    label_counter_spine = Counter(label_list_spine)
    label_counter_joint = Counter(joint_list)
    #print("label_counter_spine[13]=" + str(label_counter_spine[13]))
    #print(label_counter_spine)
    #print(label_counter)
    
    #the indexes here correspond to the labels b/c it is a dictionary
    apicals   = label_counter[2]
    basals    = label_counter[3]
    obliques  = label_counter[4]
    somas     = label_counter[5]
    axons     = label_counter[6] + label_counter[7]
    
    spines   = label_counter_spine[14]
    spine_heads    = label_counter_spine[13]
    spine_neck  = label_counter_spine[15]
    no_label     = label_counter_spine[0]
    
    apical_joint = (label_counter_joint[214],label_counter_joint[213],label_counter_joint[215],label_counter_joint[200])
    basal_joint = (label_counter_joint[314],label_counter_joint[313],label_counter_joint[315],label_counter_joint[300])
    oblique_joint = (label_counter_joint[414],label_counter_joint[413],label_counter_joint[415],label_counter_joint[400])
    soma_joint = (label_counter_joint[514],label_counter_joint[513],label_counter_joint[515],label_counter_joint[500])
    axon_joint = (label_counter_joint[614] +label_counter_joint[714],
                  label_counter_joint[613] +  label_counter_joint[713],
                  label_counter_joint[615] + label_counter_joint[715],
                  label_counter_joint[600] + label_counter_joint[700])
    
    #print((apicals, basals, obliques, somas, axons))
    #print((apical_joint + basal_joint+ oblique_joint + soma_joint + axon_joint))
    
    
    
    
    #print(np.array((apicals, basals, obliques, somas, axons)))
    synapse_total = sum((apicals, basals, obliques, somas, axons))
    synapse_total_spine = sum((spines, spine_heads, spine_neck, no_label))
    
    if synapse_total > 0:
        axon_to_vector[presyn] = np.array((apicals, basals, obliques, somas, axons))
        axon_to_vector_spine[presyn] = np.array((spines, spine_heads, spine_neck, no_label))
        axon_to_vector_spine_joint[presyn] = np.array(apical_joint + basal_joint
                                                    + oblique_joint + soma_joint + axon_joint)
    
        if synapse_total > 3:
            threshold_axon_to_vector[presyn] = np.array((apicals, basals, obliques, somas, axons))
            threshold_axon_to_vector_spine[presyn] = np.array((spines, spine_heads, spine_neck, no_label))
            threshold_axon_to_vector_spine_joint[presyn] = np.array(apical_joint + basal_joint
                                                    + oblique_joint + soma_joint + axon_joint)
    
    #print(axon_to_vector_spine)
    #break

    
axon_compartment_vector = np.array(list(axon_to_vector.values()))
threshold_axon_compartment_vector = np.array(list(threshold_axon_to_vector.values()))

axon_compartment_vector_spine = np.array(list(axon_to_vector_spine.values()))
threshold_axon_compartment_vector_spine = np.array(list(threshold_axon_to_vector_spine.values()))

axon_compartment_vector_spine_joint = np.array(list(axon_to_vector_spine_joint.values()))
threshold_axon_compartment_vector_spine_joint = np.array(list(threshold_axon_to_vector_spine_joint.values()))

#print(axon_compartment_vector_spine)

#creates the probability distribution of connections by just dividing by total
fingerprints_array = np.zeros(axon_compartment_vector.shape)
for i, axon_vector in tqdm(enumerate(axon_compartment_vector)):
    fingerprints_array[i] = axon_vector / axon_vector.sum()

#creates the probability distribution of connections by just dividing by total
threshold_fingerprints_array = np.zeros(threshold_axon_compartment_vector.shape)
for i, axon_vector in tqdm(enumerate(threshold_axon_compartment_vector)):
    threshold_fingerprints_array[i] = axon_vector / axon_vector.sum()


#creates the probability distribution of connections by just dividing by total
fingerprints_array_spine = np.zeros(axon_compartment_vector_spine.shape)
for i, axon_vector in tqdm(enumerate(axon_compartment_vector_spine)):
    fingerprints_array_spine[i] = axon_vector / axon_vector.sum()

#creates the probability distribution of connections by just dividing by total
threshold_fingerprints_array_spine = np.zeros(threshold_axon_compartment_vector_spine.shape)
for i, axon_vector in tqdm(enumerate(threshold_axon_compartment_vector_spine)):
    threshold_fingerprints_array_spine[i] = axon_vector / axon_vector.sum()
    
#creates the probability distribution of connections by just dividing by total
fingerprints_array_spine_joint = np.zeros(axon_compartment_vector_spine_joint.shape)
for i, axon_vector in tqdm(enumerate(axon_compartment_vector_spine_joint)):
    fingerprints_array_spine_joint[i] = axon_vector / axon_vector.sum()

#creates the probability distribution of connections by just dividing by total
threshold_fingerprints_array_spine_joint = np.zeros(threshold_axon_compartment_vector_spine_joint.shape)
for i, axon_vector in tqdm(enumerate(threshold_axon_compartment_vector_spine_joint)):
    threshold_fingerprints_array_spine_joint[i] = axon_vector / axon_vector.sum()


    
threshold_fingerprints_array_spine


In [ ]:
#visualize using pandas
#print(len(threshold_axon_to_vector.keys()))
#print(len(threshold_fingerprints_array_spine))
compartment_table = pd.DataFrame(threshold_fingerprints_array,
                index=threshold_axon_to_vector.keys(), 
                columns = ["apicals", "basals", "obliques", "somas", "axons"])

compartment_table


In [ ]:
#getting the axon names for the Fabian analysis
apical_frame = compartment_table.loc[compartment_table['apicals'] >= 0.75]
apical_frame
ids_apical_frame = apical_frame.index
print(ids_apical_frame)
apical_frame


In [ ]:
#visualize using pandas
#print(len(threshold_axon_to_vector.keys()))
#print(len(threshold_fingerprints_array_spine))
spine_table = pd.DataFrame(threshold_fingerprints_array_spine,
                index=threshold_axon_to_vector.keys(), 
                columns = ["spines", "spine_heads", "spine_neck", "no_label"])

spine_table



In [ ]:
#visualize using pandas

compartment_label_list = ["apicals", "basals", "obliques", "somas", "axons"]
spine_label_list = ["spines", "spine_heads", "spine_neck", "no_label"]
joint_label_list = []
for c in compartment_label_list:
    for s in spine_label_list:
        joint_label_list.append(c + ":" + s)

#print(joint_label_list)

spine_joint_table = pd.DataFrame(threshold_fingerprints_array_spine_joint,
                index=threshold_axon_to_vector_spine_joint.keys(), 
                columns = joint_label_list)

spine_joint_table

In [ ]:
###############  THE CONDENSED JOINT GROUPINGS ####################

#combine columns: so only heads and non-heads
#create mapping
compartment_label_list = ["apicals", "basals", "obliques", "somas", "axons"]
spine_label_list = ["spine_heads","spine_neck", "no_label"]
drop_spine_list = ["spines"]


mapping_merge = dict()
for i in compartment_label_list:
    label = i + ":non_head"
    for j in spine_label_list:
        if j == "spine_heads":
            mapping_merge[i + ":" + j] = i + ":" + j
        else:
            mapping_merge[i + ":" + j] = label
        
#print(mapping_merge)
        

spine_head_no_head_table = spine_joint_table.groupby(mapping_merge,axis=1).sum()


spine_head_no_head_table

In [ ]:
compartment_table

In [ ]:
#Do thresholding example for high percentage in each category

apical_axons = list(compartment_table.loc[compartment_table['apicals'] >= 0.75].index)
basal_axons = list(compartment_table.loc[compartment_table['basals'] >= 0.75].index)
oblique_axons = list(compartment_table.loc[compartment_table['obliques'] >= 0.75].index)
soma_axons = list(compartment_table.loc[compartment_table['somas'] >= 0.75].index)
axon_axons = list(compartment_table.loc[compartment_table['axons'] >= 0.75].index)

print(type(list(axon_axons)))


other_axons = list(set(list(compartment_table.index)).difference(set(apical_axons + basal_axons + oblique_axons + soma_axons + axon_axons)))
print(len(compartment_table.index))
total_axons = compartment_table.index

total_length = len(apical_axons) + len(basal_axons) + len(oblique_axons) + len(soma_axons) + len(axon_axons) + len(other_axons)
print(total_length)

print(len(other_axons))

np.savez("compartment_axon_groups.npz",
         apical_axons=apical_axons,
        basal_axons=basal_axons,
        oblique_axons=oblique_axons,
        soma_axons=soma_axons,
        axon_axons=axon_axons,
        other_axons=other_axons,
        total_axons=total_axons)





In [ ]:
Label_Table

In [ ]:
#compile the list
axon_groups = [apical_axons,basal_axons,oblique_axons,soma_axons,axon_axons,other_axons,total_axons]
axon_names = ["apical (>0.75)","basal (>0.75)","oblique (>0.75)","soma (>0.75)","axon (>0.75)","other","total"]

#pull down all of the 
postsyn_list = Label_Table.postsyn.unique()
#print(postsyn_list)
ax_group_dicts = {}

final_table = [None] * (len(postsyn_list)*len(postsyn_list) - len(postsyn_list))*len(axon_names)
counter = 0
for ax_group,ax_name in tqdm(zip(axon_groups,axon_names)):
    postsyn_axon_dict = {k:set() for k in postsyn_list}
    
    #print(postsyn_axon_dict)
    for ax in ax_group:
        #get the postsynaptic targets
        #print(ax)
        #print(Label_Table["presyn"])
        mini_table = Label_Table.loc[Label_Table["presyn"] == ax]
        if len(mini_table) > 0:
            #print(mini_table)
            unique_postsyns = mini_table.postsyn.unique()
        else:
            continue
            
        for up in unique_postsyns:
            postsyn_axon_dict[up].add(ax)
    
    ax_group_dicts[ax_name] = postsyn_axon_dict
     
#ax_group_dicts
counter = 0
#generate the rows for the table
for ax_group_name, ax_group_list in tqdm(ax_group_dicts.items()):
    #print("ax_group_name = " + str(ax_group_name))
    #print("ax_group_list = " + str(ax_group_list))
    for post,post_list in ax_group_list.items():
        for pos_post in postsyn_list:
            if pos_post == post:
                continue
            n_seg = len(post_list.intersection(ax_group_dicts[ax_group_name][pos_post]))
            #print(n_seg)
            final_table[counter] = (post,ax_group_name,pos_post,n_seg)
            #print(final_table)
            counter += 1
            
#print(final_table[(len(postsyn_list)*len(postsyn_list) - len(postsyn_list))-1])
table_without_functional = pd.DataFrame.from_records(final_table,columns= ["segment_a","connection","segment_b","n_seg_shared"])
    


            
    
    
    


In [ ]:
table_without_functional.loc[table_without_functional["connection"]=="other"]
#print(final_table[(len(postsyn_list)*len(postsyn_list) - len(postsyn_list))*5-1])
#table_without_functional

In [ ]:
#sets up the graphing variables for below
is_tuned = 'von_r2>0.05 and von_pvalue<0.05'
dori_resolution = np.pi / 2 / 4
rad2deg = 180/np.pi
bin_edges = np.linspace(0, np.pi,9)
be = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in bin_edges]])
bin_labels = list(zip(be[:-1], be[1:]))
bin_centers = np.round((bin_edges[1:] + bin_edges[:-1])/2 * rad2deg, decimals=2) 

In [ ]:
tuned_rel = nda.VonMises() & is_tuned
best_r2 = dj.U('segment_id').aggr(tuned_rel, best_r2='max(von_r2)')
pref_rel = (tuned_rel * best_r2) & 'von_r2 >= best_r2'
pref_rel


In [ ]:
#add on the functional group preferences
#restricts and appends the orientation

#query that will get the functional data
rel2 = pref_rel.proj('scan_idx',ori_1='von_pref', segment_a='segment_id') 
functional_id,ori,scan_idx = rel2.fetch("segment_a","ori_1","scan_idx") 

#create functional lookup table
func_lookup = dict()
for i,id in enumerate(functional_id):
    func_lookup[id] = [ori[i],scan_idx[i]]

#iterate through the list and only keep those whose segments are in functional list and then append the data
final_list_with_functional = []#["None"] * len(final_table)
counter = 0

dummy=2
segmentation=1


for ll in final_table: 
    if ll[0] in functional_id and ll[2] in functional_id:
        final_list_with_functional.append((ll[0],ll[1],ll[2],ll[3],
                                               int(dummy),int(segmentation),int(func_lookup[ll[0]][1]),func_lookup[ll[0]][0],
                                              func_lookup[ll[2]][0]))
        counter += 1

print(len(final_list_with_functional))

In [ ]:
#print(final_table[(len(postsyn_list)*len(postsyn_list) - len(postsyn_list))-1])
table_with_functional = pd.DataFrame.from_records(final_list_with_functional,columns= 
                ["segment_a","connection","segment_b","n_seg_shared","dummy","segmentation","scan_idx",
                        "ori_1","ori_2"])

#table_with_functional.loc[table_with_functional["n_seg_shared"]>0]
###########--------------------TABLE TO USE FOR THE FABIAN ANALYSIS------------------##########
table_with_functional

In [ ]:
table_with_functional.to_pickle("shared_connections.pkl")

In [ ]:
'''# import tsne
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import gaussian_kde
from numpy import linalg
import seaborn as sns
import random

#Try an example clustering
input_axons = threshold_fingerprints_array_spine_joint 
clusterer_input = input_axons
clusterer = KMeans(n_clusters=8, n_init=10, random_state=0)
clusterer.fit(clusterer_input)
clusterer.labels_

scores_per_cluster = np.zeros((input_axons.shape[0], clusterer.n_clusters))
for i, feature_vector in enumerate(input_axons):
    for j, center in enumerate(clusterer.cluster_centers_):
        scores_per_cluster[i, j] = np.linalg.norm(center-feature_vector)
scores_per_cluster = np.exp(-scores_per_cluster)
# scores_per_cluster = scores_per_cluster
groupings = clusterer.labels_#np.argmin(scores_per_cluster, axis=1)
groupings_dict = dict()
for i in np.unique(groupings):
    groupings_dict[i] = list()
for i, cluster in enumerate(groupings):
    groupings_dict[cluster].append(i)
mean_scores_per_group = np.zeros((clusterer.n_clusters, clusterer.n_clusters))
for i, axons in groupings_dict.items():
    group_scores = scores_per_cluster[np.array(axons)]
    for j, row in enumerate(group_scores):
        group_scores[j] /= row.sum()
    for j, column in enumerate(group_scores.T):
        mean_scores_per_group[i, j] = column.mean()
for i, row in enumerate(mean_scores_per_group):
    print("Expected Cluster:", i, "-> index of max:", np.argmax(row))
sns.heatmap(mean_scores_per_group)

'''

In [ ]:

#print the average compartment level distribution of each grouping
import numpy as np
cluster_output = clusterer.labels_

group_distributions = dict()

for i,ax in enumerate(input_axons):
    #get grouping
    group = cluster_output[i]
    if group not in group_distributions:
        group_distributions[group] = list()
        
    group_distributions[group].append(ax)

print()
#find the average distribution for each group
average_dist = [np.matrix(k).mean(axis=0) for i,k in group_distributions.items()]
variance_dist = [np.matrix(k).var(axis=0) for i,k in group_distributions.items()]
group_population = [len(k) for i,k in group_distributions.items()]

print(group_population)
print("average_dist = ")
print("apicals,     basals,   obliques,    somas,  axons")
for i in average_dist:
    print(i)
    

print("variance_dist = ")
print("apicals,     basals,   obliques,    somas,  axons")
for i in variance_dist:
    print(i)


# Fabian Analysis Code

In [ ]:
import funconnect